In [1]:
import json
import FreeSimpleGUI as sg

import matplotlib
matplotlib.use("TkAgg")

import matplotlib.pyplot as plt
import numpy as np

import clinic_extras as c

CONFIG_PATH = "config.json"


def load_config(path=CONFIG_PATH):
    defaults = {
        "NUM_MEDICOS": getattr(c, "NUM_MEDICOS", 3),
        "TAXA_CHEGADA_HORA": float(getattr(c, "TAXA_CHEGADA", 10 / 60.0)) * 60.0,
        "TEMPO_MEDIO_CONSULTA": getattr(c, "TEMPO_MEDIO_CONSULTA", 15),
        "TEMPO_SIMULACAO": getattr(c, "TEMPO_SIMULACAO", 480),
        "DISTRIBUICAO_TEMPO_CONSULTA": getattr(c, "DISTRIBUICAO_TEMPO_CONSULTA", "exponential"),
    }
    try:
        with open(path, encoding="utf-8") as f:
            cfg = json.load(f)

        if "TAXA_CHEGADA" in cfg and "TAXA_CHEGADA_HORA" not in cfg:
            cfg["TAXA_CHEGADA_HORA"] = float(cfg["TAXA_CHEGADA"]) * 60.0

        for k in defaults:
            if k in cfg:
                defaults[k] = cfg[k]
    except Exception:
        pass
    return defaults


def save_config(values, path=CONFIG_PATH):
    cfg = {
        "NUM_MEDICOS": int(values["NUM_MEDICOS"]),
        "TAXA_CHEGADA": float(values["TAXA_CHEGADA_HORA"]) / 60.0,
        "TEMPO_MEDIO_CONSULTA": float(values["TEMPO_MEDIO_CONSULTA"]),
        "TEMPO_SIMULACAO": float(values["TEMPO_SIMULACAO"]),
        "DISTRIBUICAO_TEMPO_CONSULTA": values["DISTRIBUICAO_TEMPO_CONSULTA"],
    }
    with open(path, "w", encoding="utf-8") as f:
        json.dump(cfg, f, indent=2)


def apply_to_model(values):
    c.NUM_MEDICOS = int(values["NUM_MEDICOS"])
    c.TAXA_CHEGADA = float(values["TAXA_CHEGADA_HORA"]) / 60.0
    c.TEMPO_MEDIO_CONSULTA = float(values["TEMPO_MEDIO_CONSULTA"])
    c.TEMPO_SIMULACAO = float(values["TEMPO_SIMULACAO"])
    c.DISTRIBUICAO_TEMPO_CONSULTA = values["DISTRIBUICAO_TEMPO_CONSULTA"]


def parse_inputs(values):
    out = dict(values)
    out["NUM_MEDICOS"] = int(float(values["NUM_MEDICOS"]))
    out["TAXA_CHEGADA_HORA"] = float(values["TAXA_CHEGADA_HORA"])
    out["TEMPO_MEDIO_CONSULTA"] = float(values["TEMPO_MEDIO_CONSULTA"])
    out["TEMPO_SIMULACAO"] = float(values["TEMPO_SIMULACAO"])
    out["DISTRIBUICAO_TEMPO_CONSULTA"] = values["DISTRIBUICAO_TEMPO_CONSULTA"]
    return out


# ---------------- GRÁFICOS ----------------
def graf_esperas_por_atendimento(res):
    esperas = res["tempos_espera"]
    x = list(range(len(esperas)))

    plt.figure(figsize=(9, 4.5))
    plt.plot(x, esperas)
    plt.xlabel("Ordem dos atendimentos")
    plt.ylabel("Tempo de espera (min)")
    plt.title("Evolução do tempo de espera (por doente atendido)")
    plt.grid(True)
    plt.tight_layout()
    plt.show(block=False)


def graf_lambda_vs_espera(repeticoes=20):
    resultados = []
    taxa_original = c.TAXA_CHEGADA

    for lam in range(10, 31):
        c.TAXA_CHEGADA = lam / 60.0

        esperas = []
        for _ in range(repeticoes):
            res = c.simula()
            esperas.append(res["tempo_medio_espera"])

        resultados.append((lam, float(np.mean(esperas))))

    c.TAXA_CHEGADA = taxa_original

    x = [lam for lam, _ in resultados]
    y = [v for _, v in resultados]

    plt.figure(figsize=(9, 4.5))
    plt.plot(x, y, label="Tempo médio de espera")
    plt.title("Relação entre λ e o tempo médio de espera")
    plt.xlabel("Taxa de chegada λ (doentes/hora)")
    plt.ylabel("Tempo médio de espera (min)")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show(block=False)


def graf_evolucao_fila(res):
    serie = res.get("serie_fila", [])
    tempos_h = [t / 60.0 for t, _ in serie]
    filas = [f for _, f in serie]

    plt.figure(figsize=(9, 4.5))
    plt.plot(tempos_h, filas)
    plt.xlabel("Tempo da simulação (horas)")
    plt.ylabel("Tamanho da fila")
    plt.title("Evolução do tamanho da fila ao longo do tempo")
    plt.grid(True)
    plt.tight_layout()
    plt.show(block=False)


def graf_evolucao_ocup(res):
    serie = res.get("serie_ocup", [])
    ultimo = {}
    for t, occ in serie:
        ultimo[t] = occ

    tempos = sorted(ultimo.keys())
    tempos_h = [t / 60.0 for t in tempos]
    ocup_pct = [ultimo[t] * 100 for t in tempos]

    plt.figure(figsize=(9, 4.5))
    plt.plot(tempos_h, ocup_pct)
    plt.xlabel("Tempo (horas)")
    plt.ylabel("Ocupação média (%)")
    plt.title("Ocupação média dos médicos ao longo do tempo")
    plt.ylim(0, 105)
    plt.grid(True)
    plt.tight_layout()
    plt.show(block=False)


def graf_hist_espera(res):
    esperas = res.get("tempos_espera", [])

    plt.figure(figsize=(9, 4.5))
    plt.hist(esperas, bins=20)
    plt.xlabel("Tempo de espera (min)")
    plt.ylabel("Número de doentes")
    plt.title("Distribuição dos tempos de espera")
    plt.grid(True)
    plt.tight_layout()
    plt.show(block=False)

# ---------------- APP ----------------
def main():
    sg.theme("SystemDefault")

    cfg = load_config(CONFIG_PATH)
    dist_opts = ["exponential", "normal", "uniform"]

    layout = [
        [sg.Text("App Simulação Clínica", font=("Arial", 18, "bold"), expand_x=True)],

        [sg.Frame("Config (config.json)", [
            [
                sg.Text("Médicos", size=(12, 1), font=("Arial", 13)),
                sg.Input(str(cfg["NUM_MEDICOS"]), key="NUM_MEDICOS", size=(10, 1), font=("Arial", 13)),
                sg.Text("λ (doentes/h)", size=(12, 1), font=("Arial", 13)),
                sg.Input(str(cfg["TAXA_CHEGADA_HORA"]), key="TAXA_CHEGADA_HORA", size=(10, 1), font=("Arial", 13))
            ],

            [
                sg.Text("T. médio consulta (min)", size=(18, 1), font=("Arial", 13)),
                sg.Input(str(cfg["TEMPO_MEDIO_CONSULTA"]), key="TEMPO_MEDIO_CONSULTA", size=(10, 1), font=("Arial", 13)),
                sg.Text("T. simulação (min)", size=(16, 1), font=("Arial", 13)),
                sg.Input(str(cfg["TEMPO_SIMULACAO"]), key="TEMPO_SIMULACAO", size=(10, 1), font=("Arial", 13))
            ],

            [
                sg.Text("Distribuição", size=(12, 1), font=("Arial", 13)),
                sg.Combo(
                    dist_opts,
                    default_value=cfg["DISTRIBUICAO_TEMPO_CONSULTA"],
                    key="DISTRIBUICAO_TEMPO_CONSULTA",
                    readonly=True,
                    font=("Arial", 13)
                ),
                sg.Button("Guardar JSON", key="-SAVEJSON-", font=("Arial", 13)),
                sg.Button("Recarregar JSON", key="-LOADJSON-", font=("Arial", 13))
            ]
        ], expand_x=True)],

        [sg.Frame("Ações", [
            [
                sg.Button("Correr simulação", key="-RUN-", size=(18, 1), font=("Arial", 13)),
                sg.Button("Gráfico: esperas", key="-PLOT_ESP-", size=(16, 1), disabled=True, font=("Arial", 13)),
                sg.Button("Gráfico: λ vs espera", key="-PLOT_LAM-", size=(16, 1), font=("Arial", 13)),
                sg.Button("Evolução fila", key="-PLOT_FILA-", size=(14, 1), disabled=True, font=("Arial", 13)),
                sg.Button("Evolução ocupação", key="-PLOT_OCUP-", size=(16, 1), disabled=True, font=("Arial", 13)),
                sg.Button("Histograma", key="-PLOT_HIST-", size=(12, 1), disabled=True, font=("Arial", 13)),
                sg.Button("Sair", key="-EXIT-", size=(10, 1), font=("Arial", 13))
            ]
        ], expand_x=True)],

        [sg.Frame("λ vs espera", [
            [
                sg.Text("Repetições por λ:", size=(16, 1), font=("Arial", 13)),
                sg.Input("20", key="-REP-", size=(10, 1), font=("Arial", 13))
            ]
        ], expand_x=True)],

        [sg.Frame("Resultados (última simulação)", [
            [sg.Text("Doentes atendidos:", size=(26, 1), font=("Arial", 14)), sg.Text("-", key="-DOENTES-", font=("Arial", 14))],
            [sg.Text("Tempo médio de espera (min):", size=(26, 1), font=("Arial", 14)), sg.Text("-", key="-ESPERA-", font=("Arial", 14))],
            [sg.Text("Tempo médio de consulta (min):", size=(26, 1), font=("Arial", 14)), sg.Text("-", key="-CONSULTA-", font=("Arial", 14))],
            [sg.Text("Tempo médio no sistema (min):", size=(26, 1), font=("Arial", 14)), sg.Text("-", key="-SISTEMA-", font=("Arial", 14))],
            [sg.Text("Tamanho médio da fila:", size=(26, 1), font=("Arial", 14)), sg.Text("-", key="-FILA_MEDIA-", font=("Arial", 14))],
            [sg.Text("Tamanho máximo da fila:", size=(26, 1), font=("Arial", 14)), sg.Text("-", key="-FILA_MAX-", font=("Arial", 14))],
            [sg.Text("Ocupação média:", size=(26, 1), font=("Arial", 14)), sg.Text("-", key="-OCUP_MEDIA-", font=("Arial", 14))]
        ], expand_x=True)],

        [sg.Frame("Ocupação por médico (%)", [
            [sg.Multiline("", key="-OCUP_MED-", size=(70, 8), disabled=True, expand_x=True, expand_y=True, font=("Consolas", 18))]
        ], expand_x=True, expand_y=True)],

        [sg.Frame("Métricas por médico (extra)", [
            [sg.Multiline("", key="-MET_MED-", size=(70, 8), disabled=True, expand_x=True, expand_y=True, font=("Consolas", 18))]
        ], expand_x=True, expand_y=True)],

        [sg.StatusBar("Pronto.", key="-STATUS-", expand_x=True, font=("Arial", 13))]
    ]

    scroll_layout = [[
    sg.Column(
        layout,
        scrollable=True,
        vertical_scroll_only=True,
        expand_x=True,
        expand_y=True
    )
]]



    window = sg.Window(
        "Simulação Clínica",
        scroll_layout,
        finalize=True,
        resizable=True
    )

    window.Maximize()
    resultados = None

    while True:
        event, values = window.read()
        if event in (sg.WINDOW_CLOSED, "-EXIT-"):
            break

        if event == "-SAVEJSON-":
            try:
                v = parse_inputs(values)
                save_config(v, CONFIG_PATH)
                window["-STATUS-"].update("Config guardada em config.json")
            except Exception as e:
                sg.popup_error(f"Erro ao guardar JSON: {e}")

        if event == "-LOADJSON-":
            cfg = load_config(CONFIG_PATH)
            window["NUM_MEDICOS"].update(str(cfg["NUM_MEDICOS"]))
            window["TAXA_CHEGADA_HORA"].update(str(cfg["TAXA_CHEGADA_HORA"]))
            window["TEMPO_MEDIO_CONSULTA"].update(str(cfg["TEMPO_MEDIO_CONSULTA"]))
            window["TEMPO_SIMULACAO"].update(str(cfg["TEMPO_SIMULACAO"]))
            window["DISTRIBUICAO_TEMPO_CONSULTA"].update(cfg["DISTRIBUICAO_TEMPO_CONSULTA"])
            window["-STATUS-"].update("Config recarregada do JSON")

        if event == "-RUN-":
            window["-STATUS-"].update("A correr simulação...")
            try:
                v = parse_inputs(values)
                apply_to_model(v)

                resultados = c.simula()

                window["-DOENTES-"].update(str(resultados["doentes_atendidos"]))
                window["-ESPERA-"].update(f'{resultados["tempo_medio_espera"]:.2f}')
                window["-CONSULTA-"].update(f'{resultados["tempo_medio_consulta"]:.2f}')
                window["-SISTEMA-"].update(f'{resultados["tempo_medio_sistema"]:.2f}')
                window["-FILA_MEDIA-"].update(f'{resultados["tamanho_medio_fila"]:.2f}')
                window["-FILA_MAX-"].update(str(resultados["tamanho_max_fila"]))
                window["-OCUP_MEDIA-"].update(f'{resultados["ocupacao_media"]:.3f}')

                linhas = [f"{mid}: {v*100:.2f}%" for mid, v in resultados["ocupacao_por_medico"].items()]
                window["-OCUP_MED-"].update("\n".join(linhas))

                met = resultados.get("metricas_por_medico", {})
                if met:
                    linhas2 = []
                    for mid, mm in met.items():
                        linhas2.append(
                            f"{mid}: consultas={mm.get('consultas','-')} | "
                            f"ocup={mm.get('ocupacao',0)*100:.2f}% | "
                            f"tm_consulta={mm.get('tempo_medio_consulta',0):.2f} | "
                            f"t_ocup={mm.get('tempo_total_ocupado',0):.2f}"
                        )
                    window["-MET_MED-"].update("\n".join(linhas2))
                else:
                    window["-MET_MED-"].update("(o teu motor não devolveu metricas_por_medico)")

                window["-PLOT_ESP-"].update(disabled=False)
                window["-PLOT_FILA-"].update(disabled=False)
                window["-PLOT_OCUP-"].update(disabled=False)
                window["-PLOT_HIST-"].update(disabled=False)

                window["-STATUS-"].update("Simulação concluída.")
            except FileNotFoundError:
                window["-STATUS-"].update("Erro: pessoas.json não encontrado.")
                sg.popup_error("Não encontrei pessoas.json.\nConfirma que está na mesma pasta.")
            except Exception as e:
                window["-STATUS-"].update("Erro ao correr a simulação.")
                sg.popup_error(f"Erro: {e}")

        if event == "-PLOT_ESP-" and resultados is not None:
            graf_esperas_por_atendimento(resultados)

        if event == "-PLOT_FILA-" and resultados is not None:
            graf_evolucao_fila(resultados)

        if event == "-PLOT_OCUP-" and resultados is not None:
            graf_evolucao_ocup(resultados)

        if event == "-PLOT_HIST-" and resultados is not None:
            graf_hist_espera(resultados)

        if event == "-PLOT_LAM-":
            try:
                rep = int(values["-REP-"])
                if rep <= 0:
                    raise ValueError
            except Exception:
                sg.popup_error("Repetições inválidas. Mete um inteiro > 0.")
                continue

            window["-STATUS-"].update(f"A gerar gráfico λ vs espera (repetições={rep})...")
            try:
                v = parse_inputs(values)
                apply_to_model(v)

                graf_lambda_vs_espera(repeticoes=rep)
                window["-STATUS-"].update("Gráfico gerado.")
            except Exception as e:
                window["-STATUS-"].update("Erro ao gerar gráfico.")
                sg.popup_error(f"Erro: {e}")

    window.close()


if __name__ == "__main__":
    main()



Doentes atendidos: 77
Tempo médio de espera (min): 6.22
Tempo médio de consulta (min): 13.47
Tempo médio no sistema (min): 19.7
Tamanho médio da fila: 1.37 | Máx: 7
Ocupação média: 0.831
Ocupação por médico (%):
  m0: 77.4%
  m1: 72.21%
  m2: 63.51%
Métricas por médico:
  m0: consultas=28 | ocup=77.40% | tm_consulta=13.46 min | t_ocup=376.78 min
  m1: consultas=26 | ocup=72.21% | tm_consulta=13.52 min | t_ocup=351.49 min
  m2: consultas=23 | ocup=63.51% | tm_consulta=13.44 min | t_ocup=309.18 min
